In [223]:
from fuzzywuzzy import process, fuzz
import numpy  as np
import pandas as pd
import pickle
import re

In [224]:
users_rating = pd.read_csv('../datasets/rating.csv')

movies = pd.read_csv('../datasets/movie.csv')

In [225]:
#movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27278 entries, 0 to 27277
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  27278 non-null  int64 
 1   title    27278 non-null  object
 2   genres   27278 non-null  object
dtypes: int64(1), object(2)
memory usage: 639.5+ KB


In [226]:
#users_rating.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000263 entries, 0 to 20000262
Data columns (total 4 columns):
 #   Column     Dtype  
---  ------     -----  
 0   userId     int64  
 1   movieId    int64  
 2   rating     float64
 3   timestamp  object 
dtypes: float64(1), int64(2), object(1)
memory usage: 610.4+ MB


In [227]:
#movies.isnull().sum()

movieId    0
title      0
genres     0
dtype: int64

In [228]:
# Extract year and create a new column
movies['year'] = movies['title'].str.extract(r'\((\d{4})\)')

# Remove year from 'titles' column
movies['title'] = movies['title'].str.replace(r'\(\d{4}\)', '').str.strip()

In [229]:
#movies.isnull().sum()

movieId     0
title       0
genres      0
year       22
dtype: int64

In [230]:
movies['year'] = movies['year'].fillna('')
null_id = movies[movies['year'] == '']['movieId'].tolist()
null_index = movies[movies['movieId'].isin(null_id)].index
user_index = users_rating[users_rating['movieId'].isin(null_id)].index

movies.drop(null_index, inplace=True)
users_rating.drop(user_index, inplace=True)


movies.reset_index(drop=True, inplace=True)
users_rating.reset_index(drop=True, inplace=True)


In [231]:
#movies.isnull().sum()

movieId    0
title      0
genres     0
year       0
dtype: int64

In [232]:
#movies[['title', 'year']].duplicated().sum()

16

In [233]:
duplicated_rows = movies[movies.duplicated(['title', 'year'], keep=False)]
duplicated_id = duplicated_rows['movieId'].tolist()
duplicated_index = movies[movies['movieId'].isin(duplicated_id)].index
user_index = users_rating[users_rating['movieId'].isin(duplicated_id)].index


movies.drop(duplicated_index, inplace=True)
users_rating.drop(user_index, inplace=True)

movies.reset_index(drop=True, inplace=True)
users_rating.reset_index(drop=True, inplace=True)



In [234]:
#movies[['title', 'year']].duplicated().sum()

0

In [235]:
#movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27224 entries, 0 to 27223
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  27224 non-null  int64 
 1   title    27224 non-null  object
 2   genres   27224 non-null  object
 3   year     27224 non-null  object
dtypes: int64(1), object(3)
memory usage: 850.9+ KB


In [236]:
#users_rating.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19940198 entries, 0 to 19940197
Data columns (total 4 columns):
 #   Column     Dtype  
---  ------     -----  
 0   userId     int64  
 1   movieId    int64  
 2   rating     float64
 3   timestamp  object 
dtypes: float64(1), int64(2), object(1)
memory usage: 608.5+ MB


In [237]:
movies['movieId'].duplicated().sum()

0

In [238]:
agg_ratings = users_rating.groupby('movieId').agg(mean_rating = ('rating', 'mean'), number_of_ratings = ('rating', 'count')).reset_index()

movies_rating = pd.merge(movies, agg_ratings, on='movieId', how='inner')

movies_over90 = movies_rating[movies_rating['number_of_ratings'] <= 90]

movies_id = movies_over90['movieId'].tolist()

movies_index = movies_rating[movies_rating['movieId'].isin(movies_id)].index
movie_index = movies[movies['movieId'].isin(movies_id)].index
users_index = users_rating[users_rating['movieId'].isin(movies_id)].index

movies.drop(movie_index, inplace=True)
movies_rating.drop(movies_index, inplace=True)
users_rating.drop(users_index, inplace=True)

movies.reset_index(drop=True, inplace=True)
movies_rating.reset_index(drop=True, inplace=True)
users_rating.reset_index(drop=True, inplace=True)


In [239]:
#users_rating.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19671075 entries, 0 to 19671074
Data columns (total 4 columns):
 #   Column     Dtype  
---  ------     -----  
 0   userId     int64  
 1   movieId    int64  
 2   rating     float64
 3   timestamp  object 
dtypes: float64(1), int64(2), object(1)
memory usage: 600.3+ MB


In [240]:
#movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9327 entries, 0 to 9326
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  9327 non-null   int64 
 1   title    9327 non-null   object
 2   genres   9327 non-null   object
 3   year     9327 non-null   object
dtypes: int64(1), object(3)
memory usage: 291.6+ KB


In [241]:
#movies_rating.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8794 entries, 0 to 8793
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   movieId            8794 non-null   int64  
 1   title              8794 non-null   object 
 2   genres             8794 non-null   object 
 3   year               8794 non-null   object 
 4   mean_rating        8794 non-null   float64
 5   number_of_ratings  8794 non-null   int64  
dtypes: float64(1), int64(2), object(3)
memory usage: 412.3+ KB


In [242]:
#movies_rating.head()

,movieId,title,genres,year,mean_rating,number_of_ratings
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1995,3.921240,49695
1,2,Jumanji (1995),Adventure|Children|Fantasy,1995,3.211977,22243
2,3,Grumpier Old Men (1995),Comedy|Romance,1995,3.151040,12735
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,1995,2.861393,2756
4,5,Father of the Bride Part II (1995),Comedy,1995,3.064592,12161


In [243]:
movies_rating.drop('genres', axis=1, inplace=True)
#movies_rating.head()

,movieId,title,year,mean_rating,number_of_ratings
0,1,Toy Story (1995),1995,3.921240,49695
1,2,Jumanji (1995),1995,3.211977,22243
2,3,Grumpier Old Men (1995),1995,3.151040,12735
3,4,Waiting to Exhale (1995),1995,2.861393,2756
4,5,Father of the Bride Part II (1995),1995,3.064592,12161


In [244]:
#users_rating.head()

,userId,movieId,rating,timestamp
0,1,2,3.5,2005-04-02 23:53:47
1,1,29,3.5,2005-04-02 23:31:16
2,1,32,3.5,2005-04-02 23:33:39
3,1,47,3.5,2005-04-02 23:32:07
4,1,50,3.5,2005-04-02 23:29:40


In [245]:
users_rating.drop('timestamp', axis=1, inplace=True)
#users_rating.head()

,userId,movieId,rating
0,1,2,3.5
1,1,29,3.5
2,1,32,3.5
3,1,47,3.5
4,1,50,3.5


In [246]:
movies_meta = pickle.load(open('../pickle/movies_meta.pkl','rb'))   

#movies_meta.head(5)

In [247]:
movies_meta['release_date'] = pd.to_datetime(movies_meta['release_date'], errors='coerce')
movies_meta['year'] = movies_meta['release_date'].dt.year
movies_meta['year'] = movies_meta['year'].fillna(0).astype(int).astype(str)

pickle.dump(movies_meta,open('../pickle/movies_meta.pkl','wb'))

In [248]:
movies_rating['title'] = movies_rating['title'].apply(lambda x: re.sub(r'\([^)]*\)', '', x))

In [249]:
def closest_movie(movie, movies_list):
    # Extract the year from the release date
    year = movie['year']

    # Check if the year is a valid numeric value
    try:
        year = int(year)
    except ValueError:
        # Handle the case where the year is not a valid numeric value
        print(f"Skipping movie '{movie['title']}' because the year '{year}' is not a valid numeric value.")
        return np.nan, np.nan

    # Filter movies from df2 based on the year
    movies_same_year = movies_list[movies_list['year'].astype(int) == year]

    # If there are movies in the same year, perform fuzzy matching
    if not movies_same_year.empty:
        movie_titles = movies_same_year['title'].tolist()

        closest_match = closest_title(movie['title'], movie_titles)

        if closest_match is None:
            return np.nan, np.nan

        # Get the corresponding 'release_date' for the closest match
        closest_match_info = movies_same_year.iloc[closest_match]
        title = closest_match_info['title']
        movie_id = closest_match_info['id']

        # Drop movie after it is found
        movie_index = movies_list[(movies_list['id'] == movie_id)].index
        movies_list.drop(movie_index, inplace=True)
        movies_list.reset_index(drop=True, inplace=True)

        return str(movie_id), title
    else:
        return np.nan, np.nan


def closest_title(query, choices):
    # Preprocess the strings
    query = preprocess_string(query)
    choices = [preprocess_string(choice) for choice in choices]

    # Use process.extractOne to find the best match
    match, score = process.extractOne(query, choices, scorer=fuzz.token_sort_ratio)

    return choices.index(match) if score >= 80 else None

# Function to preprocess strings before matching
def preprocess_string(s):
    # Check if s is a string
    if isinstance(s, str):
        # Remove non-alphanumeric characters
        s = ''.join(char for char in s if char.isalnum() or char.isspace())
        return s.strip()
    else:
        # If s is not a string (e.g., NaN), return an empty string or handle it accordingly
        return ''


In [250]:
print("Running... (may take a couple of minutes)")

movies_rating[['id', 'title']] = movies_rating.apply(lambda x: pd.Series(closest_movie(x, movies_meta)), axis=1)

#movies_rating.head(5)

,movieId,title,year,mean_rating,number_of_ratings,id
0,1,Toy Story,1995,3.921240,49695,862
1,2,Jumanji,1995,3.211977,22243,8844
2,3,Grumpier Old Men,1995,3.151040,12735,15602
3,4,NaN,1995,2.861393,2756,NaN
4,5,Father of the Bride Part II,1995,3.064592,12161,11862


In [251]:
#movies_rating.isnull().sum()

movieId                 0
title                3086
year                    0
mean_rating             0
number_of_ratings       0
id                   3086
dtype: int64

In [252]:
null_id = movies_rating[movies_rating['id'].isnull()]['movieId'].tolist()
null_index = movies_rating[movies_rating['movieId'].isin(null_id)].index
user_index = users_rating[users_rating['movieId'].isin(null_id)].index

movies_rating.drop(null_index, inplace=True)
users_rating.drop(user_index, inplace=True)

movies_rating.reset_index(drop=True, inplace=True)
users_rating.reset_index(drop=True, inplace=True)

In [253]:
#movies_rating.isnull().sum()

movieId              0
title                0
year                 0
mean_rating          0
number_of_ratings    0
id                   0
dtype: int64

In [254]:
#movies_rating.head()

,movieId,title,year,mean_rating,number_of_ratings,id
0,1,Toy Story,1995,3.921240,49695,862
1,2,Jumanji,1995,3.211977,22243,8844
2,3,Grumpier Old Men,1995,3.151040,12735,15602
3,5,Father of the Bride Part II,1995,3.064592,12161,11862
4,6,Heat,1995,3.834930,23899,949


In [255]:
#users_rating.head()

,userId,movieId,rating
0,1,2,3.5
1,1,29,3.5
2,1,32,3.5
3,1,47,3.5
4,1,50,3.5


In [256]:
movies_rating.drop(['mean_rating', 'number_of_ratings'], axis=1, inplace=True)

new_users_rating = pd.merge(users_rating, movies_rating, on='movieId', how='inner')

movies_rating.drop(['movieId'],  axis=1, inplace=True)

new_users_rating.drop(['movieId', 'title', 'year'],  axis=1, inplace=True)

In [257]:
#movies_rating.head()

,title,year,id
0,Toy Story,1995,862
1,Jumanji,1995,8844
2,Grumpier Old Men,1995,15602
3,Father of the Bride Part II,1995,11862
4,Heat,1995,949


In [258]:
#new_users_rating.head() 

,userId,rating,id
0,1,3.5,8844
1,5,3.0,8844
2,13,3.0,8844
3,29,3.0,8844
4,34,3.0,8844


In [259]:
pickle.dump(movies_rating,open('../pickle/movies_rating.pkl','wb'))
pickle.dump(new_users_rating,open('../pickle/users_rating.pkl','wb'))

print("Done")